# Backdoored Webserver


```{image} images/backdoored_webserver.png
:alt: backdoored_webserver
:width: 400px
:align: center
```

## Introduction

In this section, we will discuss the implementation of a backdoored web server in C. The backdoored web server is a simple web server that listens on a specified port and serves files from a specified directory. The backdoor functionality allows an attacker to execute arbitrary commands on the server by sending specially crafted HTTP requests.

## Explanation of Code by ChatGPT 4o (OpenAI)

The provided C code implements a basic HTTP server with a backdoor functionality that listens for incoming HTTP requests, processes them, and executes commands embedded in the URL of the request if specific conditions are met. Here's a detailed explanation of the code:

---

### **Key Functionalities**
1. **Socket Communication:**
   - The server listens on a specified port for incoming connections.
   - It processes HTTP requests from clients and sends appropriate responses.

2. **Backdoor Command Execution:**
   - If the URL in the HTTP request contains `/exec/`, the server interprets the string after `/exec/` as a system command.
   - This command is executed using `popen`, and the output is sent back to the client.

3. **URL Decoding:**
   - Encoded URLs are decoded to retrieve their original form.

4. **Response Handling:**
   - The server can send different HTTP responses: 200 OK, 404 Not Found, or 400 Bad Request.

---

### **Main Components**
#### **Global Definitions**
- **Buffer Sizes:**
  `BUFFER_SIZE` and other constants are used to define limits for reading data and constructing responses.
- **Predefined Responses:**
  Strings for various HTTP response codes (`200 OK`, `404 Not Found`, etc.) are constructed.

#### **Main Function**
1. **Socket Setup:**
   - A socket is created and configured to listen on the specified port (`socket` and `bind` calls).
   - The `setsockopt` function allows reusing the socket address.

2. **Listening for Connections:**
   - The `listen` function queues incoming connections.
   - The server accepts connections with `accept` and processes each request in a loop.

3. **Processing Requests:**
   - **Receiving Data:**
     Incoming HTTP requests are read into a buffer using `recv`.
   - **URL Parsing:**
     The `get_URL_from_header` function extracts the URL from the HTTP request header.
   - **URL Decoding:**
     The extracted URL is decoded using `URL_decode`.
   - **Command Handling:**
     - If the URL contains `/exec/`, the part after it is treated as a shell command.
     - The command is executed, and its output is sent to the client as part of the HTTP response.
   - **Invalid Requests:**
     Requests without `/exec/` or malformed URLs receive appropriate error responses (`404` or `400`).

4. **Response Sending:**
   - `send_response` sends the constructed response back to the client.

#### **Helper Functions**
1. **`send_response`:**
   Sends an HTTP response back to the client socket, handling both successful and error cases.

2. **`get_URL_from_header`:**
   Extracts the URL from the first line of the HTTP request header.

3. **`URL_decode`:**
   Decodes percent-encoded characters in URLs (e.g., `%20` becomes a space).

4. **`does_URL_contain_slash_exec`:**
   Checks if the URL contains the `/exec/` substring.

5. **`get_CMD_from_decoded_URL`:**
   Extracts the command from the decoded URL after `/exec/`.

6. **`execute_command_return_output`:**
   Executes a shell command using `popen` and captures its output.

---

### **Key Vulnerabilities and Concerns**
1. **Backdoor Implementation:**
   - The `/exec/` functionality allows remote execution of arbitrary shell commands, creating a severe security vulnerability.

2. **Unsafe Command Execution:**
   - Commands are executed without validation or sanitization, exposing the server to command injection attacks.

3. **Lack of Authentication:**
   - No mechanism to authenticate or authorize incoming requests, making the server open to exploitation.

4. **Memory Leaks and Errors:**
   - The program lacks comprehensive error handling for `malloc` and `realloc`, potentially leading to memory leaks.

5. **Debug Information Exposure:**
   - Printing debug information, such as the request content and decoded commands, can expose sensitive data.

6. **Non-Threaded Design:**
   - The server handles only one request at a time, which is inefficient for multiple clients.

---

### **Recommendations**
- Remove or restrict the `/exec/` functionality to prevent unauthorized command execution.
- Implement input validation and sanitization.
- Add authentication mechanisms to ensure only authorized users can interact with the server.
- Use secure coding practices, such as properly managing memory allocations and handling errors.
- Improve the server's architecture to support concurrent connections, possibly using threading or asynchronous I/O.

---

This code appears to be designed for demonstration or testing purposes rather than production use due to its significant security flaws.

## Program Code

```{literalinclude} C_Code/backdoored_webserver.c
```

## Compile the Code

In [1]:
import os
curr_dir = ""
curr_dir = os.getcwd()

code_dir = curr_dir + "/" + "C_Code"
os.chdir(code_dir)

In [2]:
exec_status = os.system("gcc -ggdb3 -o backdoored_webserver backdoored_webserver.c")

## Run the Code

Running the code starts a webserver that listens on the specified port as an argument. You can access the server by sending HTTP requests to the server's IP address and port.  In this example, we will start the server on port 8080 like so:

```bash
./backdoored_webserver 8080
```

The output is as follows:

```
The number of command line arguments is -> 1
The command line argument is 8080
```

In another terminal, we use `curl` to send an HTTP request to the server.  The curl command is as follows:

```bash
curl -i http://127.0.0.1:8080
```

and the response is:

```
HTTP/1.1 404 Not Found
```

However, if we send a request with the `/exec/` keyword, we can execute commands on the server. For example:

```bash
curl -i http://127.0.0.1:8080/exec/whoami
```

The response will be:

```
HTTP/1.1 200 OK

haxor
```

Another example:

```bash
curl -i http://127.0.0.1:8080/exec/ls
```

The response will be:

```
HTTP/1.1 200 OK

backdoored_webserver
backdoored_webserver.c
```